# Issue #10 Problem with Primary Beam at different frequencies

edited by JC, Martin, Louise

The issue seems to be solved, Jean implemented the scaling of the primary beam with the frequency. At 220, we know it is not a good approximation so we print some warnings. 

In [ ]:
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(16,4))
plt.rc('font',size=12)
plt.rc('text',usetex=False)
plt.rc('image', cmap='viridis')

import os

path = os.getcwd()
path_mm = os.path.dirname(path)
import sys
sys.path.append(path_mm)

import numpy as np
import qubic
import instrument as instr


In [ ]:
# This is the analytical synthesized for a pixel at the focal plane center 
# without accounting for the primary beam (just the mukltple peaks)
def sb_noprim(th_deg, nu):
    th = np.radians(th_deg)
    lam = 3e8/nu
    P = 20
    deltax = 0.013
    df = 300.
    abscissa = np.pi * deltax/lam * th
    sb = np.sin(P*abscissa)**2 / np.sin(abscissa)**2
    return sb/np.max(sb)

In [ ]:
# With Multiband = False  => using a QubicInstrument
d = qubic.qubicdict.qubicDict()
d.read_from_file('global_source_oneDet.dict')

d['multiband'] = False

# Test with 150 or 220 GHz 
# At 220 you should use FI and not TD
freqs = np.array([210*1e9, 220*1e9, 230*1e9])
d['config'] = 'FI'
# freqs = np.array([1.32638868e+11, 1.50398340e+11, 1.67001321e+11])

nn = 1000
th = np.linspace(-30, 30, nn)
ph = 0

fig, axs = plt.subplots(3, 1, figsize=(20, 15))
axs = np.ravel(axs)

beams = ['gaussian', 'fitted_beam', 'multi_freq']

for j, ibeam in enumerate(beams):
    print('\n Computing beam model: {}'.format(ibeam))
    allbeams = np.zeros((len(freqs), nn))
    d['beam_shape'] = ibeam
    ax1 = axs[j]
#     ax1 = axs[2*j]
#     ax2 = axs[2*j+1]
    for i in range(len(freqs)):
        d['filter_nu'] = freqs[i]
        q = instr.QubicInstrument(d)
        allbeams[i, :] = q.primary_beam(np.radians(np.abs(th)), ph)

        p = ax1.plot(th, allbeams[i,:],'--', label='nu={0:6.1f}'.format(freqs[i]/1e9))
        ax1.plot(th, allbeams[i,:]*sb_noprim(th, freqs[i]), color=p[0].get_color())
        ax1.set_title(ibeam)#'d[beam_shape]='+d['beam_shape']+' , Multiband='+str(d['multiband']))
        ax1.set_xlim(-20, 20)
        ax1.legend()
        
#         mask = np.where((th<-7) & (th>-9))
#         if i == 2: 
#             ax2.axhline(np.max(allbeams[i,mask]*sb_noprim(th[mask], freqs[i])), c='g' )
#         p = ax2.plot(th, allbeams[i,:],'--', label='nu={0:6.1f}'.format(freqs[i]/1e9))
#         ax2.plot(th, allbeams[i, :]*sb_noprim(th, freqs[i]), color=p[0].get_color())
#         ax2.set_xlim(-12, -7)
#         ax2.set_title(ibeam)#'d[beam_shape]='+d['beam_shape']+' , Multiband='+str(d['multiband']))
#         ax2.grid()

In [ ]:
q.primary_beam??

In [ ]:
# With multiband = True code have to be written differently
d = qubic.qubicdict.qubicDict()
d.read_from_file('global_source_oneDet.dict')

d['multiband'] = True

# Test with 150 or 220 GHz 
# At 220 you should use FI and not TD
d['config'] = 'FI'
d['filter_nu'] = 220e9 

q = qubic.QubicMultibandInstrument(d)
nsub = len(q)
nus = np.zeros(nsub)
for i in range(nsub): 
    nus[i] = q[i].filter.nu

indices = [0, 6, 11]

beams = ['gaussian', 'fitted_beam', 'multi_freq']

fig, axs = plt.subplots(3, 1, figsize=(12, 14))
axs = np.ravel(axs)
print('============== Starting the loop ===================')
for b, ibeam in enumerate(beams):
    print('Computing beam model: {}'.format(ibeam))
    d['beam_shape'] = ibeam
    q = qubic.QubicMultibandInstrument(d)
    ax = axs[b]
    for i, ind in enumerate(indices):
        print(ind)
        allbeams[i, :] = q[ind].primary_beam(np.radians(np.abs(th)),ph)
        p = ax.plot(th, allbeams[i, :], '--', 
                    label='nu={0:6.1f}'.format(nus[ind]/1e9))
        ax.plot(th, allbeams[i, :] * sb_noprim(th, freqs[i]), 
                color=p[0].get_color())

        ax.set_title('d[beam_shape]=' + ibeam + ' , Multiband=' + str(d['multiband']))
        ax.legend()